In [1]:
import os
import sys
os.chdir('/home/docadmin/YQ/spider/RegulatoryNews/')
#sys.path.insert(0, "//home//docadmin//ZackC//NegNews//")
#sys.path.insert(0, "//home//docadmin//YQ//spider//RegulatoryNews//") 
#from nn_config.configg import *  # coinfig includes names, domain_list, scrapy_dir, 
#import config as config 
import config as config
from scrapyd_driver_v2 import *
#from newsapi_downloader import *

import os
import pickle
import time
from scrapyd_api import ScrapydAPI
from time import sleep
#import subprocess
import datetime
import shutil
import jsonlines as jl

import dateparser
from dateutil import parser
import numpy as np
import pandas as pd
import copy

from nn_base.nn_classes import search_entity as SE
from nn_base.nn_textprocess import OrigTxt_PreProcess
from nn_base.nn_webprocess import window_search_regex
from nn_base.nn_pdf2 import create_pdf_summary
from nn_base.nn_textprocess import force_near_filter
from nn_base.nn_translate import translate_searchterms

from nn_base.nn_textprocess import entity_match_score
from nn_base.nn_textprocess import entity_stem_match
from nn_base.nn_textprocess import clean_paragraph
from nn_base.nn_textprocess import LSA_function
from nn_base.nn_textprocess import normalize_LSA
from nn_base.nn_textprocess import rerank_searchresults
from nn_base.nn_textprocess import summary_scores
from nn_base.nn_textprocess import OrigTxt_PreProcess
from nn_base.nn_textprocess import w2v_SortOutput
from nn_base.nn_textprocess import w2v_se2
import pymagnitude

## Regulatory Spider + Newsapi Spider

In [2]:
###Website Collection###
scrapyd = ScrapydAPI('http://localhost:6800')
run_rn_spider(scrapyd,entity_list=config.names,max_searching_pages=config.max_rg_searching_pages,domain_list=config.domain_list,num_pages=config.num_pages)
#newsapi_downloader(output_dir=config.output_dir,entity_list=config.names,domain_list=config.domain_list)
wait(scrapyd)
collect_jl(config.output_dir)

In [3]:
wait(scrapyd)
collect_jl(config.output_dir)

## Previous Code

In [ ]:
###Website Collection###
scrapyd = ScrapydAPI('http://localhost:6800')
run_rn_spider(scrapyd,names)
newsapi_downloader(output_dir=config.output_dir,entity_list=config.names,domain_list=config.domain_list)
wait(scrapyd)
collect_jl(config.output_dir)

In [23]:
def format_date(dt):
    try:
        return dateparser.parse(dt).strftime('%m-%d-%Y')
    except:
        return None

In [39]:
###reading and preprocess jason lines###
files=os.listdir(config.output_dir)
files=[file for file in files if len(file)>5]
file=files[np.argmax(list(map(lambda x:dateparser.parse(x[:19].replace('.',':')),files)))]
jl_file_path=config.output_dir+file+'/'+'all_rn_results_'+file[:10]+'.jl'
json_list=[]
with jl.open(jl_file_path,'r') as reader:
    for jdic in reader:
        json_list.append(jdic)
df_jl=pd.DataFrame(json_list)
columnsTitles = ['url','source','entity','date','title','content','summary','url']
df_jl['summary']=df_jl['summary'].apply(lambda x: ''.join(map(str, x)).strip())
df_jl=df_jl[df_jl['summary']!='None']
df_jl['date']=df_jl['date'].apply(format_date)
df_jl['content']=df_jl.apply(lambda x: x['content'] if x['content']!='' else x['summary'],axis=1)
df_jl['title_content']=df_jl.apply(lambda x: str(x['title'])+' '+x['content'] ,axis=1)
#df_jl['bf_pdf_text']=df_jl.apply(lambda x : window_search_regex(x['title_content'],'',x['entity'],entity=0) ,axis=1) ###qykqykqyk config.entity

In [91]:
se_list=[]
for iden,entity in enumerate(config.names):
    df_=df_jl[df_jl['entity']==entity]
    
    #identify fname and lname (to be removed eventually)
    name  = entity
    fname = name.split()[0] if config.entity == 0 else name 
    lname = name.split()[1] if len(name.split()) > 1 & config.entity==0 else ''
    
    searchterm_list, _  = translate_searchterms(config.searchterms2) if config.termlist==2 else translate_searchterms(config.searchterms1)
    query_list = ', '.join(list(set([x for x in list(df_['source'])])))
    entity_querylist = query_list

    se = SE(search_lang = config.search_lang
        , searchid = str(iden)
        , fname = fname
        , lname=lname
        , mname=''
        , city=''
        , state=''
        , state2=''
        , country=''
        , occupation=''
        , company = ''
        , company2= ''
        , querylist=query_list
        , searchtermlist=searchterm_list
       )
    
    #Attributes about what we are searching for  
    se.search_lang_short=se.search_lang
      
    #Attributes about returned result
    se.querylisturl[0]=query_list #[source for source in df_['source']] 
    se.entity_querylist=query_list 
    
    urllist=df_['url']
    se.origtextlist[0]=dict(zip(df_['url'],df_['title_content']))
    
    se.lastsearchdate=[dict(zip(df_['url'],df_['date']))]
    #extracted_text = extract_web_results(v, self.se.entity_fname, self.se.entity_lname, config.entity)
    #self.se.textsummary[idx][k] = extracted_text#'test'#str(generate_summary(extract_web_results_three(v),n_sentence=1)) #summary of results 
                
    se.textsummary=[dict(zip(df_['url'],df_['summary']))]#####Zack to review if this makes sense
    se.textlist[0]=df_['title_content']
    se.NewsSource=[df_['source']]
 ###   ###########self.pdftext= OrigText_preprocess
    
    #Attributes about matching
    #2) apply a near filter to it if config parameter is >= 0
            #note: if lname is not present this filter forces the 'entity' or fname simply to be found near anything(' ')
	    #print('Total Texts Before Filter: ',len(textlist))
    if config.near_filter_distance >= 0:
        textlist =df_.apply(lambda x:force_near_filter(x['content'],se.entity_fname, se.entity_lname, config.near_filter_distance, config.entity),axis=1) 
    
    #3) clean up the texts using clean_paragraph function
    textlist=list(map(lambda x:window_search_regex(x,se.entity_fname,se.entity_lname,entity=config.entity),textlist))
    textlist , origtext = OrigTxt_PreProcess(textlist, se.entity_fname, se.entity_lname, delim ='<!@&>', window=False)
    #textlist , origtext = OrigTxt_PreProcess(df_['bf_pdf_text'], se.entity_fname, se.entity_lname, delim ='<!@&>', window=False)
    textlist_clean=copy.deepcopy(textlist)
    textlist_zip = list(zip(textlist, origtext, urllist))
    textlist_clean = [x[0] for x in textlist_zip if len(x[0]) != 0] #first element in ziped textlist
    se.pdftext[0] = [x[1] for x in textlist_zip if len(x[0]) != 0] #first element in ziped textlist
    urllist = [x[2] for x in textlist_zip if len(x[0]) != 0] #second element in zipped textlist
    #4) Calculate relevancy scores(THESE NEED TO BE UPDATED)
    scoreslist = [entity_match_score(text, se, 0) for text in textlist_clean]
    #5) Find stem matches
    stemmatchlist = [entity_stem_match(' '.join(text).split(), se, 0) for text in textlist_clean]
    
    #6) Save the processed results (cleaned text, fuzzyscores, details, and matched stems) into appropriate se slot
    se.urllist[0] = urllist
    se.textlist[0] = textlist_clean
    se.list_fuzzyscore[0] = [np.mean([x for x in scores if x > 0]) if len([x for x in scores if x > 0])>0 else 0 for scores in scoreslist]
    se.list_fuzzyscoredetail[0] = scoreslist
    se.list_matchedstems[0] = stemmatchlist
    print('urls:',len(se.urllist[0]),'orig:',len(se.textlist[0]),'pdf:',len(se.pdftext[0]))
    print('urls:',len(urllist),'orig:',len(textlist_clean),'pdf:',len(se.pdftext[0]))
    
    for idx, text in enumerate(se.textlist):
        if text == []: se.urllist[idx]=[]
         
    searchresults = ' '.join(['{}:{}'.format(x,len(y)) for (x,y) in zip(se.search_lang, se.urllist)])
    print('urls:',len(se.urllist[0]),'orig:',len(se.textlist[0]),'pdf:',len(se.pdftext[0]))
    print('Searched {} languages total. The results are: {}'.format(se.search_lang_ct, searchresults))    

    wv = pymagnitude.Magnitude(config.word_embedding)

    #Execute W2V and transform sentences into sentences of embeddings 
    se = w2v_se2(se, wv, entity=config.entity ,ndoc=config.search_results, nsent=1000, sent_max =config.max_sent , min_sim = 0.1, delim = '')
                  
    print('========================THIS IS THE SORTED LIST======================')
    print(list(zip(se.urllist[0],se.textlist[0])))
    #print(self.se.w2vInfo[0])

    se = w2v_SortOutput(se, config.entity, max_sent = config.max_sent)
    print(se.w2vDfSorted[0])
    se = LSA_function(se)
    se = rerank_searchresults(se, config.LSA_score_filter)
    se = summary_scores(se, config.search_results)
    se_list.append(se)
    
    print('exporting results to pickle, which can then be loaded statically for testing new funcitonality using load_results() in text_processing')
    pickle.dump(se, open(config.pdf_path + se.entity_fname + ' ' + se.entity_lname+ ' ' + time.strftime("%m-%d-%Y") + '.pk', 'wb'))

    #fname: filename of pdf created , failed: list of languages that failed to create a pdf 
    fname, failed = create_pdf_summary(se) 
    print('PDF file stored at - {}'.format(os.path.basename(fname)))
    #for x in failed:
    #    print('PDF creation failed on language - {}'.format(x))
    ##jsonname = os.path.splitext(fname)[0]+'.json'
    ##dataout = se.to_json(jsonname)
    ##if dataout == 0: #to_json() will return a 0 if sucsesful 
    ##    print('Data saved at {}'.format(os.path.basename(jsonname)))
    ##else: 
    ##    print('Data Not Saved')
    #print('total chars translated {}'.format(transcount))
    
    
    
    

urls: 38 orig: 38 pdf: 38
urls: 38 orig: 38 pdf: 38
urls: 38 orig: 38 pdf: 38
Searched 1 languages total. The results are: english:38
0:00:02.929965
========================THIS IS THE SORTED LIST======================
[('http://www.finra.org/about/firms-we-regulate/firms-we-regulate-n', ['east plano parkway suite plano tx nolan securities corp main road monterey ma nomura securities international inc worldwide plaza west th street new york ny norfolk markets llc th ave th floor new york ny north bridge capital llc constitution lane suite c danvers ma mailing address constitution lane suite c danvers ma north capital private securities corporation e ft union blvd suite salt lake city ut north inlet advisors llc one morrocroft centre morrison blvd suite charlotte nc north point advisors llc california street suite san francisco ca north ridge securities corp walt whitman rd melville ny north sea securities l']), ('http://www.finra.org/newsroom/2014/finra-bars-broker-insider-trading-japa

THIS IS THE LENGTH OF W2vDoc 34
URL LIST BEFORE FILTER:  38
[(1.0, 0.786657175831803, 0.6871398275765368, 0.6644918404331916, 0.662879517259241, 0.6115674018949728, 0.6019416028976046, 0.5729970867107744, 0.4863601849939476, 0.48458818444774193, 0.47921270394165577, 0.4781662727940034, 0.4765960457299035, 0.46437922229467393, 0.45785238527191746, 0.4479564879190173, 0.4479564879190173, 0.44331708279810517, 0.43534651671558366, 0.4290416340308774, 0.405399005403421, 0.3655505436272837, 0.3332072809914339, 0.32676989317998306, 0.3257548810489162, 0.29747133557323885, 0.24892355130951144, 0.24110003408488218, 0.2377926452406469, 0.23623736883907892, 0.22817932884650968, 0.19777374928239755, 0.056241037695486135, 0.0, -9999, -9999, -9999, -9999), ('https://www.bloomberg.com/news/articles/2018-02-05/nomura-said-to-hire-brevan-s-mohideen-once-fired-amid-libor', 'https://www.bloomberg.com/news/articles/2018-06-14/nomura-calls-for-intervention-to-snap-india-bonds-vicious-cycle', 'http://money.

/usr/local/lib/python3.5/dist-packages/docx/styles/styles.py:54: UserWarning: style lookup by style_id is deprecated. Use style name as key instead.
  warn(msg, UserWarning)


KeyError: 0

In [81]:
a=se_list[1]

In [89]:
a.te

[(['tyler austin miles the park former director of operations and the water park itself were charged last week with involuntary manslaughter according to an indictment unsealed friday',
   'all five defendants are also charged with aggravated battery and aggravated child endangerment for injuries suffered by others including four children on the waterslide before caleb schwab was killed in',
   'cnn marlena baldacci sheena jones and hollie silverman contributed to this report'],
  ['finra s complaint charges that both provisions violate finra rules concerning language or conditions that firms may place in customer agreements',
   'finra s complaint seeks an expedited hearing because schwab s conduct is ongoing as the firm has continued to use account agreements containing these provisions in opening more than new customer accounts since october the issuance of a disciplinary complaint represents the initiation of a formal proceeding by finra in which findings in the complaint have not 

In [9]:
#creat pdf
create_pdf_summary(se_list[0])

IndexError: list index out of range